In [1]:
import os

In [2]:
%pwd  #project working directory

'e:\\Data Science\\LoanApprovalAppWith-Mlflow\\Loan-APP\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Data Science\\LoanApprovalAppWith-Mlflow\\Loan-APP'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_dir: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from Loan_APP.constants import *
from Loan_APP.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_dir=config.source_dir,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
        

In [8]:
import os
import urllib.request as request
import zipfile
from Loan_APP import logger
from Loan_APP.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
           filename, headers = request.urlretrieve(
                url= self.config.source_dir,
                filename= self.config.local_data_file
            )
           logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-25 19:07:24,319: INFO: common: yaml file: config\config.yaml is loaded successfully!]
[2025-02-25 19:07:24,325: INFO: common: yaml file: params.yaml is loaded successfully!]
[2025-02-25 19:07:24,330: INFO: common: yaml file: schema.yaml is loaded successfully!]
[2025-02-25 19:07:24,334: INFO: common: created directory at: artifacts]
[2025-02-25 19:07:24,338: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-25 19:07:26,065: INFO: 1377390207: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 7846
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e95f191fc191536e6462c798dbe6b1eeb2670547fd48abffa1652471989d2f8b"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E28A:30863F:CB7C:1951B:67BDC795
Accept-Ranges: bytes
D